In [41]:
import pandas as pd
classes_df = pd.read_csv("raw_data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edgelist_df = pd.read_csv("raw_data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
features_df = pd.read_csv("raw_data/elliptic_bitcoin_dataset/elliptic_txs_features.csv")

In [42]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [43]:
# Prepare node features
features = features_df.iloc[:, 1:].values
node_ids = features_df.iloc[:, 0].values

# Create node ID to index mapping
node_id_to_idx = {node_id: idx for idx, node_id in enumerate(node_ids)}

# Prepare labels - CORRECT MAPPING: '1'=illicit, '2'=licit
# Map to: 0=unknown, 1=illicit, 2=licit
classes_df['class'] = classes_df['class'].map({'unknown': 0, '1': 1, '2': 2})
labels = np.zeros(len(node_ids), dtype=int)
for _, row in classes_df.iterrows():
    if row['txId'] in node_id_to_idx:
        labels[node_id_to_idx[row['txId']]] = row['class']

# Prepare edge index
edge_list = []
for _, row in edgelist_df.iterrows():
    if row['txId1'] in node_id_to_idx and row['txId2'] in node_id_to_idx:
        edge_list.append([node_id_to_idx[row['txId1']], node_id_to_idx[row['txId2']]])

edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()

# Convert to PyTorch tensors
x = torch.tensor(features, dtype=torch.float)
y = torch.tensor(labels, dtype=torch.long)

print(f"Number of nodes: {x.shape[0]}")
print(f"Number of features: {x.shape[1]}")
print(f"Number of edges: {edge_index.shape[1]}")
print(f"Label distribution: Unknown={sum(y==0)}, Illicit={sum(y==1)}, Licit={sum(y==2)}")

Number of nodes: 203768
Number of features: 166
Number of edges: 234353
Label distribution: Unknown=157204, Illicit=4545, Licit=42019


In [44]:
# Create train/test split (only on labeled nodes)
labeled_mask = y > 0
labeled_indices = torch.where(labeled_mask)[0].numpy()

train_indices, test_indices = train_test_split(
    labeled_indices, 
    test_size=0.3, 
    random_state=42, 
    stratify=y[labeled_indices].numpy()
)

# Create masks
train_mask = torch.zeros(len(y), dtype=torch.bool)
test_mask = torch.zeros(len(y), dtype=torch.bool)
train_mask[train_indices] = True
test_mask[test_indices] = True

print(f"Training nodes: {train_mask.sum()}")
print(f"Test nodes: {test_mask.sum()}")
print(f"Unknown nodes: {(~labeled_mask).sum()}")

Training nodes: 32594
Test nodes: 13970
Unknown nodes: 157204


In [45]:
class GCNFraudDetector(torch.nn.Module):
    def __init__(self, in_features=166, hidden=64, out_classes=3, dropout=0.5):
        super().__init__()
        self.gcn1 = GCNConv(in_features, hidden)
        self.gcn2 = GCNConv(hidden, hidden // 2)
        self.fc = torch.nn.Linear(hidden // 2, out_classes)
        self.dropout = dropout
    
    def forward(self, x, edge_index):
        # GCN Layer 1
        x = self.gcn1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # GCN Layer 2
        x = self.gcn2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Output layer
        x = self.fc(x)
        return x

model = GCNFraudDetector(in_features=x.shape[1], hidden=64, out_classes=3, dropout=0.5)
print(model)

GCNFraudDetector(
  (gcn1): GCNConv(166, 64)
  (gcn2): GCNConv(64, 32)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)


In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = x.to(device)
edge_index = edge_index.to(device)
y = y.to(device)
train_mask = train_mask.to(device)
test_mask = test_mask.to(device)

# Calculate class weights to handle imbalance
# Only consider labeled classes (1=illicit, 2=licit) since class 0 (unknown) is not in training
train_labels = y[train_mask]
n_illicit = (train_labels == 1).sum().float()
n_licit = (train_labels == 2).sum().float()
total = train_labels.shape[0]

# Compute weights: inversely proportional to class frequency
# We need weights for all 3 classes (0, 1, 2) even though 0 is not in training
weight_illicit = total / (2.0 * n_illicit)  # Higher weight for minority class
weight_licit = total / (2.0 * n_licit)      # Lower weight for majority class
class_weights = torch.tensor([1.0, weight_illicit, weight_licit], dtype=torch.float).to(device)

print(f"Using device: {device}")
print(f"\nClass distribution in training set:")
print(f"  Class 1 (Illicit):  {n_illicit.int()} samples ({n_illicit/total*100:.1f}%)")
print(f"  Class 2 (Licit):    {n_licit.int()} samples ({n_licit/total*100:.1f}%)")
print(f"\nClass weights:")
print(f"  Class 0 (Unknown): {class_weights[0]:.4f} (not used in training)")
print(f"  Class 1 (Illicit):  {class_weights[1]:.4f}")
print(f"  Class 2 (Licit):    {class_weights[2]:.4f}")
print(f"\nWeight ratio (Illicit/Licit): {class_weights[1]/class_weights[2]:.2f}x")

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

Using device: cpu

Class distribution in training set:
  Class 1 (Illicit):  3181 samples (9.8%)
  Class 2 (Licit):    29413 samples (90.2%)

Class weights:
  Class 0 (Unknown): 1.0000 (not used in training)
  Class 1 (Illicit):  5.1232
  Class 2 (Licit):    0.5541

Weight ratio (Illicit/Licit): 9.25x


In [47]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = criterion(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test(mask):
    model.eval()
    out = model(x, edge_index)
    pred = out.argmax(dim=1)
    correct = pred[mask] == y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc, pred[mask].cpu(), out[mask].cpu()

# Training loop
print("Training GCN...")
for epoch in range(1, 51):
    loss = train()
    
    if epoch % 10 == 0:
        train_acc, _, _ = test(train_mask)
        test_acc, _, _ = test(test_mask)
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

print("\nTraining completed!")

Training GCN...
Epoch 010, Loss: 0.5411, Train Acc: 0.8127, Test Acc: 0.8134
Epoch 020, Loss: 0.4826, Train Acc: 0.8168, Test Acc: 0.8169
Epoch 030, Loss: 0.4240, Train Acc: 0.8490, Test Acc: 0.8477
Epoch 040, Loss: 0.3980, Train Acc: 0.8715, Test Acc: 0.8709
Epoch 050, Loss: 0.3689, Train Acc: 0.8719, Test Acc: 0.8731

Training completed!


In [48]:
# Final evaluation
_, test_pred, test_out = test(test_mask)
test_labels = y[test_mask].cpu().numpy()

# Convert to binary classification (0=licit, 1=illicit)
# Model outputs: 0=unknown, 1=illicit, 2=licit
binary_pred = (test_pred.numpy() == 1).astype(int)  # 1 if illicit, 0 otherwise
binary_labels = (test_labels == 1).astype(int)  # 1 if illicit, 0 otherwise

# Calculate metrics
accuracy = accuracy_score(binary_labels, binary_pred)
precision = precision_score(binary_labels, binary_pred, zero_division=0)
recall = recall_score(binary_labels, binary_pred, zero_division=0)
f1 = f1_score(binary_labels, binary_pred, zero_division=0)

# Get probabilities for AUC-ROC and PR-AUC (probability of illicit class)
test_probs = F.softmax(test_out, dim=1)[:, 1].numpy()  # Column 1 = illicit
auc_roc = roc_auc_score(binary_labels, test_probs)
pr_auc = average_precision_score(binary_labels, test_probs)

# RMSE
rmse = np.sqrt(mean_squared_error(binary_labels, binary_pred))

print("\n" + "="*50)
print("FINAL EVALUATION METRICS")
print("="*50)
print(f"Accuracy:  {accuracy*100:.2f}%")
print(f"Precision: {precision*100:.2f}%")
print(f"Recall:    {recall*100:.2f}%")
print(f"F1 Score:  {f1*100:.2f}%")
print(f"AUC-ROC:   {auc_roc:.4f}")
print(f"PR-AUC:    {pr_auc:.4f}")
print(f"RMSE:      {rmse:.4f}")
print("="*50)


FINAL EVALUATION METRICS
Accuracy:  87.31%
Precision: 42.42%
Recall:    83.94%
F1 Score:  56.36%
AUC-ROC:   0.9408
PR-AUC:    0.7212
RMSE:      0.3563
